In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt

import IPython.display as ipd

In [2]:
import os
import time

import tensorflow as tf
import numpy as np

from hparams import create_hparams
import data
import utils
import commons
import waveglow as model

In [3]:
model_dir = "./logs/vq_1_shallow/"

hparams = create_hparams(model_dir, [], initialize=False)
utils.check_git_hash(model_dir)

INFO:tensorflow:The value of 'n_checkpoints' is '5', but is overwritten by '20'.
INFO:tensorflow:The value of 'tfr_dir' is 'datasets/ljs', but is overwritten by '/data/project/rw/datasets/ljs'.
INFO:tensorflow:The value of 'model_dir' is 'logs/vq_1_shallow', but is overwritten by './logs/vq_1_shallow/'.
INFO:tensorflow:------------------- All configurations --------------------
INFO:tensorflow:    n_early_size = 2
INFO:tensorflow:    max_wav_value = 32768.0
INFO:tensorflow:    fmax = 8000.0
INFO:tensorflow:    ema_decay = 0.9999
INFO:tensorflow:    learning_rate = 0.0001
INFO:tensorflow:    bottleneck_dims_per_bit = 16
INFO:tensorflow:    sigma = 1.0
INFO:tensorflow:    train_files = filelists/ljs_audio_text_train_filelist.txt
INFO:tensorflow:    fft_size = 1024
INFO:tensorflow:    kernel_size = 3
INFO:tensorflow:    adam_epsilon = 1e-08
INFO:tensorflow:    exponential_moving_average = False
INFO:tensorflow:    model_dir = ./logs/vq_1_shallow/
INFO:tensorflow:    clip_gradients = None


In [4]:
input_fn = data.InputPipeline(hparams, tf.estimator.ModeKeys.EVAL)
itr = input_fn().make_one_shot_iterator()

features = itr.get_next()[0]

c = features[hparams["local_condition"]]
g = features[hparams["global_condition"]] if hparams["global_condition"] is not None else None

# type casting
if hparams.ftype != tf.float32:
  c = tf.cast(c, hparams.ftype)

with tf.variable_scope("model", custom_getter=commons.weight_norm_getter):
  wn = model.WaveGlow(hparams, tf.estimator.ModeKeys.PREDICT)
  sigma = tf.placeholder(hparams.ftype, ())
  
  predictions = wn.infer(features={"c": c, "g": g}, sigma=sigma)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [5]:
checkpoint = tf.train.latest_checkpoint(model_dir)
saver = tf.train.Saver()
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from ./logs/vq_1_shallow/model.ckpt-1000000


In [7]:
c_np, wav = sess.run([c, predictions], feed_dict={sigma: 1.})
wav = wav[0,:,0].astype(np.float32)
ipd.Audio(wav, rate=hparams.sample_rate)
# if global condition is on
# g_np, c_np, wav = sess.run([g, c, predictions], feed_dict={sigma: 1.})
# wav = wav[0,:,0].astype(np.float32)
# print(g_np)
# ipd.Audio(wav, rate=hparams.sample_rate)

In [ ]:
# if global condition is on, try voice conversion
# m: 241, 245, 254, 284, 292, 326, 360, 376
# f: 249, 265, 266, 273, 297, 303, 305, 312, 318
wav = sess.run(predictions, feed_dict={g: [303], c: c_np, sigma: 1.0})
wav = wav[0,:,0].astype(np.float32)
ipd.Audio(wav, rate=hparams.sample_rate)